In [ ]:
#pip install pyts

In [ ]:
#pip install sktime

In [8]:
import numpy as np
import pyts
from pyts.metrics import dtw
import sklearn

In [24]:
datasets = [
"ETCHING_Multivar",
]

datasets_path = "../datasets"

for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    Dataset = Dataset
    print(Dataset.shape)
    
    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")
    Labels = Labels.squeeze()
    print(Labels.shape)

(128, 112, 18)
(128,)


In [25]:
Dataset.shape

(128, 112, 18)

In [26]:
Dataset1 = Dataset[:,:,0].copy()

In [27]:
Dataset1.shape

(128, 112)

In [28]:
##Generate the initial training and test sets to feed to the model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Dataset1, Labels, train_size=0.8 ,shuffle=True, random_state=123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((102, 112), (26, 112), (102,), (26,))

## Run the normal DTW on the first dimension

In [29]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
classifier = KNeighborsTimeSeriesClassifier(distance= 'dtw', n_jobs=10)

In [30]:
classifier.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier(n_jobs=10)

In [31]:
print("Test Accuracy:{:.2f}".format(classifier.score(X_test, y_test)))

Test Accuracy:0.77


## Run the DTW-I on the all dimensions

In [33]:
X_train, X_test, y_train, y_test = train_test_split(Dataset, Labels, train_size=0.8 ,shuffle=True, random_state=123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((102, 112, 18), (26, 112, 18), (102,), (26,))

In [34]:
def DTW_custom_metric(Dataset):
    distance_matrix = np.zeros((Dataset.shape[0], Dataset.shape[0]))
    for i in range(Dataset.shape[0]):
        for j in range(i, Dataset.shape[0]):
            distance = 0
            for m in range(Dataset.shape[-1]):
                dist = pyts.metrics.dtw(x=Dataset[i,:,m], y=Dataset[j,:,m],dist='square', method='sakoechiba')
                distance += dist
            distance_matrix[i,j] = distance
            distance_matrix[j, i] = distance
    return distance_matrix

In [35]:
dist_matrix = DTW_custom_metric(X_train)
dist_matrix.shape

(102, 102)

In [36]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1, metric='precomputed')

In [37]:
classifier.fit(DTW_custom_metric(X_train), y_train)

KNeighborsClassifier(metric='precomputed', n_neighbors=1)

In [38]:
def DTW_test_distance(X_test, X_train):
    distance_matrix = np.zeros((X_test.shape[0], X_train.shape[0]))
    for i in range(X_test.shape[0]):
        for j in range(X_train.shape[0]):
            distance = 0
            for m in range(X_test.shape[-1]):
                dist = pyts.metrics.dtw(x=X_test[i,:,m], y=X_train[j,:,m],dist='square', method='fast')
                distance += dist
            distance_matrix[i,j] = distance
    return distance_matrix

In [39]:
y_pred = classifier.predict(DTW_test_distance(X_test, X_train))
y_pred.shape

(26,)

In [40]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9230769230769231


## Run with multivariate capability of sktime

In [41]:
Dataset2 = np.swapaxes(Dataset, 1,2)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(Dataset2, Labels, train_size=0.8 ,shuffle=True, random_state=123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((102, 18, 112), (26, 18, 112), (102,), (26,))

In [43]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
classifier = KNeighborsTimeSeriesClassifier(distance= 'dtw', n_jobs=10)
classifier.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier(n_jobs=10)

In [44]:
print("Test Accuracy:{:.2f}".format(classifier.score(X_test, y_test)))

Test Accuracy:0.92


## TWE

In [45]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
classifier = KNeighborsTimeSeriesClassifier(distance= 'twe', n_jobs=10)
classifier.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier(distance='twe', n_jobs=10)

In [46]:
print("Test Accuracy:{:.2f}".format(classifier.score(X_test, y_test)))

Test Accuracy:0.92


In [ ]:
dist = DTW_custom_metric(X1=Dataset[0,:,:], X2=Dataset[1,:,:])
dist

In [ ]:
dist1 = dtw(x=Dataset[0,:,0], y=Dataset[1,:,0],
           dist='square', method='classic', return_cost=True)
dist1